# Búsqueda semántica
Autor: Eric S. Tellez <eric.tellez@infotec.mx>


# Language models
En los word embeddings, cada palabra del vocabulario tiene asignado un vector denso de manera estática, fruto de su semántica basada en la hipótesis distribucional. Los modelos de lenguaje van más allá, intentando no solo tener en cuenta una palabra para el vector, si no que el vector mismo es dependiente del contexto, por lo cual puede desambiguar de manera natural palabras idénticas (homónimos) usando dicha información contextual. Adicionalmente, capturan información relevante de grandes corpus de texto, aportando muchas veces información de un mundo (i.e., si se entrenan usando la wikipedia, tendrán información relavante de multiples dominios).

La punta de lanza de los modelos de lenguaje es el aprendizaje profundo, y más precisamente, con el uso de _transformers_. Uno de los modelos de lenguaje más utilizado es BERT [@VSPU2017]. La persona interesada más sobre ellos <https://huggingface.co/>. 
Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

En lo que resta de este notebook se estarán viendo como usar los modelos, y más precisamente los embeddings para realizar tareas de recuperación de información.


In [1]:
using Pkg
Pkg.activate(".")


  Activating project at `~/Cursos/IR-2024/Unidades`


In [2]:
using SimilaritySearch, LinearAlgebra, HypertextLiteral, CodecZlib, JSON, Random, StatsBase, DataFrames
using Downloads: download
using Transformers
using Transformers.TextEncoders
using Transformers.HuggingFace

In [3]:
textencoder, bertmodel = hgf"dccuchile/bert-base-spanish-wwm-cased"

(BertTextEncoder(
├─ TextTokenizer(MatchTokenization(WordPieceTokenization(bert_cased_tokenizer, WordPiece(vocab_size = 31002, unk = [UNK], max_char = 100)), 5 patterns)),
├─ vocab = Vocab{String, SizedArray}(size = 31002, unk = [UNK], unki = 4),
├─ startsym = [CLS],
├─ endsym = [SEP],
├─ padsym = [PAD],
├─ trunc = 512,
└─ process = Pipelines:
  ╰─ target[token] := TextEncodeBase.nestedcall(string_getvalue, source)
  ╰─ target[token] := Transformers.TextEncoders.grouping_sentence(target.token)
  ╰─ target[(token, segment)] := SequenceTemplate{String}([CLS]:<type=1> Input[1]:<type=1> [SEP]:<type=1> (Input[2]:<type=2> [SEP]:<type=2>)...)(target.token)
  ╰─ target[attention_mask] := (NeuralAttentionlib.LengthMask ∘ Transformers.TextEncoders.getlengths(512))(target.token)
  ╰─ target[token] := TextEncodeBase.trunc_and_pad(512, [PAD], tail, tail)(target.token)
  ╰─ target[token] := TextEncodeBase.nested2batch(target.token)
  ╰─ target[segment] := TextEncodeBase.trunc_and_pad(512, 1, tail, t

In [34]:
function encode_text_(bertmodel, e)
    s = bertmodel(e)
    p = vec(s.pooled)
    normalize!(p)
end

function encode_text(textencoder, bertmodel, text::AbstractString)
    e = encode(textencoder, text)
    encode_text_(bertmodel, e)
end

function encode_text(textencoder, bertmodel, text::AbstractVector)
    length(text) == 2 || throw(ArgumentError("the model supports a sentence and a pair of sentences"))
    e = encode(textencoder, [text])
    encode_text_(bertmodel, e)
end

function encode_corpus(textencoder, bertmodel, corpus::AbstractVector)
    # we avoid using the Bert / Beto API directly to reduce memory consumption
    # nonetheless, it will make use of more dynamic allocations
    n = length(corpus)
    E = Matrix{Float32}(undef, 768, n)
    for (i, text) in enumerate(corpus)
        if rand() < 0.1
            @info i, text
        end
        E[:, i] .= encode_text(textencoder, bertmodel, text)
    end

    E
end

function create_index(db, recall)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
    optimize!(index, MinRecall(recall))
    index
end

create_index (generic function with 1 method)

In [33]:
encode_text(textencoder, bertmodel, ["¿Cuántos zapatos tiene María?", "que pasa con el elefante", "cuantos zapatos tiene Maria", "santa maria de los buenos aires"])

768-element Vector{Float32}:
 -0.014974176
 -0.03792605
  0.002626905
  0.050325755
 -0.048552472
 -0.029904956
  0.04216009
 -0.078885645
 -0.026764376
 -0.029372465
  0.02667069
 -0.002202712
  0.03841882
  ⋮
 -0.0727948
 -0.01639543
 -0.061481655
  0.006171282
 -0.04631392
  0.0053181024
 -0.0091918195
  0.016311778
  0.046157602
  0.022465635
  0.0061214804
 -0.02674574

In [35]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")

D, Q = DataFrame.(read_news())
DataFrame(D), DataFrame(Q)
#@show Set(unique(D.labels)) == Set(unique(Q.labels))


<h1>Cargando el corpus</h1>

(30244×2 DataFrame
   Row │ corpus                             labels          
       │ String                             String          
───────┼────────────────────────────────────────────────────
     1 │ RT @SanGonAzc: @RadioIndeMexico …  AdriDelgadoRuiz
     2 │ “El periodismo serio y responsab…  AdriDelgadoRuiz
     3 │ "...Sí esperamos que se aumente …  AdriDelgadoRuiz
     4 │ RT @Univ_Cultura: Encuentran sis…  El_Universal_Mx
     5 │ Los tacos son grandes embajadore…  CNNEE
     6 │ Esta es la lista de países a los…  NTN24
     7 │ Por desgracia, esta tendencia im…  UniNoticias
     8 │ La embajadora de Ucrania en Esta…  TelemundoNews
     9 │ Dos Bocas alista motores | La re…  SinEmbargoMX
    10 │ Analizamos el papel de las mujer…  Reforma
    11 │ Ignasi Aballí: «Cometo más error…  abc_es
   ⋮   │                 ⋮                         ⋮
 30235 │ RT @EFEnoticias_ES: Olona formal…  EFEnoticias
 30236 │ #LoMásLeído 📰 Muller-Weiss, la …  abc_es
 30237 │ RT @Univ_Cultura

In [ ]:
@time C = encode_corpus(textencoder, bertmodel, D.corpus)

[ Info: (25, "#Hilo 🪡 #Revive una #trilogía sobre el momento político histórico de 1988 con nuestros #documentales (en inglés): #CÁRDENAS OPPORTUNITY. POWER. DESOLATION ▶️ https://t.co/iRN0kyqZU1 | #MAQUÍO REVELLION. SEDUCTION. TRAGEDY ▶️ https://t.co/ovA7VsxkEd https://t.co/HQx3NJqUI0")
[ Info: (29, "#Opinión | ¿Cómo pudieron tantas personas entrar a México y desplazarse luego hasta la frontera con EU? La respuesta es obvia: con la ayuda de organizaciones criminales especializadas en el contrabando de migrantes, dice @ruizhealy.\nhttps://t.co/a8yBxdJJom https://t.co/dFlEyQRJoo")
[ Info: (31, "Tiger Woods anuncia que jugará otro torneo a pesar de los malos resultados en el Masters de Augusta\nhttps://t.co/FmN1kqYxJU")
[ Info: (33, "“Soy saxofonista no violinista”: María Elena atacada con ácido corrige a Sergio Mayer y lo llama oportunista https://t.co/nP98RbiJIB https://t.co/qw6XHqdUXD")
[ Info: (48, "🎥 Los Rams se coronan campeones del Super Bowl LVI: fanáticos celebran en el centro d

In [ ]:
C

### Se crea el índice métrico

In [ ]:
@time index = create_index(MatrixDatabase(C), 0.95)

### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad (~42k consultas)

In [ ]:
let k = 11
    Qvectors = MatrixDatabase(vectorize_corpus_as_prototypes(Q.corpus, E, T))
    t1 = @elapsed I, _ = searchbatch(index, Qvectors, k)
    # WARNING: Don't run the following line, it takes too much time
    ex = ExhaustiveSearch(; db=index.db, dist=index.dist)
    t2 = @elapsed gI, _ = searchbatch(ex, Qvectors, k)
    r = macrorecall(gI, I)
    n = size(I, 2)
    labels = String.(D.labels)
    s1 = scores(Q.labels, knn(I, labels))
    s2 = scores(Q.labels, knn(gI, labels))
    
    @htl """
    <div>(brute vs indexed) macro-recall: $r, n: $n</div>
    <div>searchgraph search time: $t1, scores: $s1</div>
    <div>brute force search time: $t2, scores: $s2</div>
    """
end

### Búsqueda y presentación de los resultados

In [ ]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = zeros(Float32, size(E.X, 1))
    vectorize_as_prototype!(q, qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.labels[id])</td><td> $(D.corpus[id])</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


In [ ]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


In [ ]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.corpus))
        search_and_display(index, D.corpus[qid], 7, D, E, T)
    end
end

# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Implemente su búsqueda semántica. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.
